In [1]:
import xretrieval

xretrieval.list_datasets()

['coco-val-2017']

In [2]:
dataset = xretrieval.load_dataset("coco-val-2017")

2024-11-20 12:12:57.988 | INFO     | xretrieval.datasets.coco:__init__:26 - COCO validation dataset found in data/coco/val2017, skipping download


In [3]:
dataset

,image_id,file_name,image_path,caption,name
0,139,000000000139.jpg,data/coco/val2017/000000000139.jpg,A woman stands in the dining area at the table...,"book,chair,clock,dining table,microwave,person..."
1,285,000000000285.jpg,data/coco/val2017/000000000285.jpg,A big burly grizzly bear is show with grass in...,bear
2,632,000000000632.jpg,data/coco/val2017/000000000632.jpg,"Bedroom scene with a bookcase, blue comforter ...","bed,book,chair,potted plant"
3,724,000000000724.jpg,data/coco/val2017/000000000724.jpg,A stop sign is mounted upside-down on it's pos...,"car,stop sign,truck"
4,776,000000000776.jpg,data/coco/val2017/000000000776.jpg,"Three teddy bears, each a different color, snu...","bed,teddy bear"
...,...,...,...,...,...
4995,581317,000000581317.jpg,data/coco/val2017/000000581317.jpg,A woman holding a small item in a field. Woman...,"cell phone,person"
4996,581357,000000581357.jpg,data/coco/val2017/000000581357.jpg,Skate boarder hang time with young man appeari...,"bench,person,skateboard"
4997,581482,000000581482.jpg,data/coco/val2017/000000581482.jpg,A fancy metal cloth is seen against huge barre...,clock
4998,581615,000000581615.jpg,data/coco/val2017/000000581615.jpg,A picture of a white toilet in the rest room. ...,toilet


In [4]:
import xinfer

xinfer.list_models("blip2")

                              Available Models                              
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Implementation ┃ Model ID                          ┃ Input --> Output    ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ transformers   │ Salesforce/blip2-opt-6.7b-coco    │ image-text --> text │
│ transformers   │ Salesforce/blip2-flan-t5-xxl      │ image-text --> text │
│ transformers   │ Salesforce/blip2-opt-6.7b         │ image-text --> text │
│ transformers   │ Salesforce/blip2-opt-2.7b         │ image-text --> text │
│ transformers   │ sashakunitsyn/vlrm-blip2-opt-2.7b │ image-text --> text │
└────────────────┴───────────────────────────────────┴─────────────────────┘

In [5]:
model = xinfer.create_model("Salesforce/blip2-opt-2.7b", device="cuda", dtype="float16")

2024-11-20 12:12:59.852 | INFO     | xinfer.models:__init__:63 - Model: Salesforce/blip2-opt-2.7b
2024-11-20 12:12:59.853 | INFO     | xinfer.models:__init__:64 - Device: cuda
2024-11-20 12:12:59.853 | INFO     | xinfer.models:__init__:65 - Dtype: float16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)


In [7]:
dataset

Dataset({
    features: ['image_id', 'file_name', 'image_path', 'caption', 'name'],
    num_rows: 5000
})

In [8]:
def generate_captions_batch(examples):
    image_paths = [filename for filename in examples['image_path']]
    prompts = [""] * len(image_paths) 
    
    results = model.infer_batch(image_paths, prompts, max_new_tokens=20)
    captions = [res.text for res in results]

    examples['blip2_caption'] = captions
    return examples

In [9]:
blip2_captioned_dataset = dataset.map(generate_captions_batch, batched=True, batch_size=200, desc="Generating captions")
blip2_captioned_dataset


Generating captions:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset({
    features: ['image_id', 'file_name', 'image_path', 'caption', 'name', 'blip2_caption'],
    num_rows: 5000
})

In [10]:
blip2_captioned_dataset.to_pandas()

,image_id,file_name,image_path,caption,name,blip2_caption
0,139,000000000139.jpg,data/coco/val2017/000000000139.jpg,A woman stands in the dining area at the table...,"book,chair,clock,dining table,microwave,person...",a living room with a fireplace and a table
1,285,000000000285.jpg,data/coco/val2017/000000000285.jpg,A big burly grizzly bear is show with grass in...,bear,a brown bear sitting on the grass
2,632,000000000632.jpg,data/coco/val2017/000000000632.jpg,"Bedroom scene with a bookcase, blue comforter ...","bed,book,chair,potted plant","a bedroom with a bed, a dresser, a bookcase an..."
3,724,000000000724.jpg,data/coco/val2017/000000000724.jpg,A stop sign is mounted upside-down on it's pos...,"car,stop sign,truck",a stop sign on a street corner
4,776,000000000776.jpg,data/coco/val2017/000000000776.jpg,"Three teddy bears, each a different color, snu...","bed,teddy bear",a group of three teddy bears
...,...,...,...,...,...,...
4995,581317,000000581317.jpg,data/coco/val2017/000000581317.jpg,A woman holding a small item in a field. Woman...,"cell phone,person",a woman standing on a hill
4996,581357,000000581357.jpg,data/coco/val2017/000000581357.jpg,Skate boarder hang time with young man appeari...,"bench,person,skateboard",a man on a skateboard
4997,581482,000000581482.jpg,data/coco/val2017/000000581482.jpg,A fancy metal cloth is seen against huge barre...,clock,a large clock in a large building with a large...
4998,581615,000000581615.jpg,data/coco/val2017/000000581615.jpg,A picture of a white toilet in the rest room. ...,toilet,a urinal in a bathroom
